<a href="https://colab.research.google.com/github/DenisseUrenda/MachineLearning/blob/main/L09_Metamodelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metodos de ensamble

https://scikit-learn.org/stable/modules/ensemble.html#ensemble

El objetivo de los metodos de ensable es combinar las predicciones de varios estimadores base construidofs con un algoritmo de aprendizaje dado.

Hay dos familias de metodos de ensamble:
- En los **metodos de ponderado** el principio fundamental es construir varios estimadores de forma independiente y luego promediar sus predicciones.
- En contraste, los **metodos de boosting**, los estimadores base son construidos secuencialmente y uno intenta reducir el sesgo del estimador combinado. La idea principal es combinar varios modelos bebiles para producir un ensamble poderoso.

## Bagging

En los algoritmos de ensamble, los metodos de bagging forman una clase de algoritmos que construyen varias instancias de un estimador en subconjuntos aleatorios del conjunto de entrenamineto original y luego sus predicciones individuales son anadidas para formar una prediccion final.

En `scikit-learn`, los metodos de bagging pueden ser encontrados como un meta-estimador unificado llamado `BaggingClassifier` (o `BaggingRegressor` para regresion). Este toma como entrada un estimator especificado por el usuario y parametros que especifican la estrategia para crear subconjuntos aleatorios:

- `max_samples`, `max_features`: controla el tamano de los subconjuntos
- `bootstrap`, `bootstrap_features`: controla si las observaciones y variables son tomadas con o sin reemplazo.
- `oob_score=True`: cuando es usado un subconjunto de observaciones la precision generalizada puede ser estimada con las observacionens en el conjunto "out-of-bag".


In [ ]:
# Lectura de datos
import pandas as pd

path = 'https://gist.githubusercontent.com/javierIA/7d8564f02ae92bf722800465da2d0f7a/raw/6e0b018296ba1bafe2f20319c9f3e3a8573b151e/salarios.csv'
salarios = pd.read_csv(path)
salarios.head(n=10)

,edad,estudio,horas,clase_salario,valor_salario
0,27,9,40,0,12967.330
1,30,9,40,0,20171.400
2,42,9,40,1,39051.040
3,68,9,25,1,33567.250
4,34,8,40,0,8247.420
5,26,13,50,0,6769.345
6,23,4,25,0,2266.799
7,20,10,40,0,17190.410
8,21,10,15,0,2254.370
9,41,10,40,0,13464.160


In [ ]:
# Transformacion de datos
from sklearn.preprocessing import MinMaxScaler

salarios.iloc[:, :3] = MinMaxScaler().fit_transform(salarios.iloc[:, :3])
salarios.head(n=10)

,edad,estudio,horas,clase_salario,valor_salario
0,0.158730,0.533333,0.549296,0,12967.330
1,0.206349,0.533333,0.549296,0,20171.400
2,0.396825,0.533333,0.549296,1,39051.040
3,0.809524,0.533333,0.338028,1,33567.250
4,0.269841,0.466667,0.549296,0,8247.420
5,0.142857,0.800000,0.690141,0,6769.345
6,0.095238,0.200000,0.338028,0,2266.799
7,0.047619,0.600000,0.549296,0,17190.410
8,0.063492,0.600000,0.197183,0,2254.370
9,0.380952,0.600000,0.549296,0,13464.160


In [ ]:
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

datos = salarios.iloc[:,0:3]     # Variables predictoras
clases = salarios.clase_salario  # Variable de respuesta para clasificacion
valor = salarios.valor_salario   # Variable de respuesta para regresion

clasificador = KNeighborsClassifier(n_neighbors=3)
regresor = KNeighborsRegressor(n_neighbors=3)

In [ ]:
datos

,edad,estudio,horas
0,0.158730,0.533333,0.549296
1,0.206349,0.533333,0.549296
2,0.396825,0.533333,0.549296
3,0.809524,0.533333,0.338028
4,0.269841,0.466667,0.549296
...,...,...,...
95,0.238095,0.800000,0.760563
96,0.047619,0.600000,0.338028
97,0.460317,0.133333,0.549296
98,0.317460,0.533333,0.619718


In [ ]:
n_modelos = 50

meta_clasificador = BaggingClassifier(
    estimator=clasificador, n_estimators=n_modelos, max_samples=0.3
).fit(datos, clases)
meta_regresor = BaggingRegressor(
    estimator=regresor, n_estimators=n_modelos, max_samples=0.3
).fit(datos, valor)

In [ ]:
from sklearn.metrics import accuracy_score, r2_score

print(f'Accuracy: {accuracy_score(clases, meta_clasificador.predict(datos))}')
print(f'R2: {r2_score(valor, meta_regresor.predict(datos))}')

Accuracy: 0.84
R2: 0.329912785759362
